In [1]:
import pandas as pd
import sqlite3
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets.embed import embed_minimal_html
#Label, BoundedIntText, IntText, Valid, Dropdown, Select, Text, Textarea, Combobox, Password, Image, Button, DatePicker, Tab

In [2]:
#!pip install ipython-sql

In [4]:
#!pip install widgetsnbextension 
#!pip install ipywidgets 
#!pip install voila
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [3]:
class FridgeDB(object):
    
    def __init__(self):
        flag = True

        self.fridge_text = widgets.IntText(
            value="1",
            description='Fridge:',
            disabled=False
        )

        self.shelf_text = widgets.IntText(
            value="1",
            description='Shelf:',
            disabled=False
        )

        self.rack_text = widgets.IntText(
            value="1",
            description='Rack:',
            disabled=False
        )

        self.layer_text = widgets.IntText(
            value="1",
            description='Layer:',
            disabled=False
        )

        self.box_text = widgets.IntText(
            value="1",
            description='Box:',
            disabled=False
        )

        self.row_text = widgets.IntText(
            value='1',
            description='Row:',
            disabled=False
        )

        self.column_text = widgets.IntText(
            value="1",
            description='Column:',
            disabled=False
        )
    
        self.id_text = widgets.Text(
            placeholder='Enter Sample ID',
            description='Sample ID:',
            disabled=False
        )

        self.description_text = widgets.Textarea(
            placeholder='Enter Description of Sample',
            description='Description:',
            disabled=False
        )
        
        self.find_Button = widgets.Button(
            description='Find Sample',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Finds Sample',
            icon='check' # (FontAwesome names without the `fa-` prefix)
        )
        
        self.insert_Button = widgets.Button(
            description='Add Sample',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Adds Sample',
            icon='check' # (FontAwesome names without the `fa-` prefix)
        )
        
        self.remove_Button = widgets.Button(
            description='Remove Sample',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Removes Sample',
            icon='check' # (FontAwesome names without the `fa-` prefix)
        )
        
        self.edit_Button = widgets.Button(
            description='Save Edit',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Save Edit',
            icon='check' # (FontAwesome names without the `fa-` prefix)
        )
        
        self.fetch_Button = widgets.Button(
            description='Get Information',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='get more info',
            icon='check' # (FontAwesome names without the `fa-` prefix)
        )
        
        self.add_equipment_Button = widgets.Button(
            description='Add new fridge',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Add Fridge',
            icon='check' # (FontAwesome names without the `fa-` prefix)
        )
        
        self.find_Button.on_click(self.confirm_click)
        self.insert_Button.on_click(self.confirm_click)
        self.remove_Button.on_click(self.confirm_click)
        self.edit_Button.on_click(self.confirm_click)
        self.add_equipment_Button.on_click(self.confirm_click)
        
        self.output = widgets.Output(layout={'border': '1px solid black'})
        
        self.find_gui = widgets.VBox(children = (self.id_text, self.fridge_text,self.shelf_text,self.rack_text,self.layer_text,self.box_text,self.row_text,self.column_text,self.find_Button,self.output),
                 layout = widgets.Layout(border = 'solid 2px gray', width = '900px')
                )
        self.insert_gui = widgets.VBox(children = (self.id_text, self.fridge_text,self.shelf_text,self.rack_text,self.layer_text,self.box_text,self.row_text,self.column_text,self.description_text,self.insert_Button,self.output),
                 layout = widgets.Layout(border = 'solid 2px gray', width = '900px')
                )
        self.remove_gui = widgets.VBox(children = (self.id_text, self.fridge_text,self.shelf_text,self.rack_text,self.layer_text,self.box_text,self.row_text,self.column_text,self.remove_Button,self.output),
                 layout = widgets.Layout(border = 'solid 2px gray', width = '900px')
                )
        self.edit_gui = widgets.VBox(children = (self.id_text, self.fetch_Button,self.fridge_text,self.shelf_text,self.rack_text,self.layer_text,self.box_text,self.row_text,self.column_text,self.description_text,self.edit_Button,self.output),
                 layout = widgets.Layout(border = 'solid 2px gray', width = '900px')
                )
        self.add_equipment_gui = widgets.VBox(children = (self.shelf_text,self.rack_text,self.layer_text,self.box_text,self.row_text,self.column_text,self.add_equipment_Button,self.output),
                 layout = widgets.Layout(border = 'solid 2px gray', width = '900px')
                )

        self.tab = widgets.Tab()
        self.tab.children = [self.find_gui,self.insert_gui,self.remove_gui,self.edit_gui,self.add_equipment_gui]
        self.tab.set_title(0, 'Find Sample')
        self.tab.set_title(1, 'Add Sample')
        self.tab.set_title(2, 'Remove Sample')
        self.tab.set_title(3, 'Edit Sample')
        self.tab.set_title(4, 'Add New Fridge')
        
    def confirm_click(self,b):
        global flag
        flag = False
        self.output.clear_output()
        
        with self.output:
            if self.id_text.value:
                %sql sqlite:///equipment.db
                result = %sql SELECT * FROM equipment
                print(result)
            else:
                print('false')
        
    def get_gui(self):
        display(self.tab)
        embed_minimal_html('export.html', views=[self.tab], title='Widgets export')
            
if __name__ == '__main__':
    test = FridgeDB()
    test.get_gui()
    
    